In [4]:
import sqlite3 
connect = sqlite3.connect("D:/AHN/Codng/python/2025_kimmich.kr_web/coding/DB/money.db")
cursor = connect.cursor()

In [43]:
# 현재 지금 어떤 것과 연결되어있는지 확인하기
res = cursor.execute("SELECT name FROM sqlite_master")
print(res.fetchone())

('money',)


In [75]:
from datetime import date
today = date.today()

cursor.execute(f"""
    INSERT INTO 'money' VALUES
        ('{today}', 'out',10003,'축구')
""")
connect.commit()

In [76]:
res = cursor.execute("SELECT * FROM money;")
rows = res.fetchall()

for row in rows:
    print(row)

('2025-05-12', 'in', '1000', None)
('2025-05-12', 'in', '1000', None)
('2025-05-12', 'in', '10001', None)
('2025-05-12', 'out', '10001', None)
('2025-05-12', 'out', '10002', None)
('2025-05-12', 'out', '10003', None)
('2008', 'out', '10003', '축구')
('2025-05-12', 'out', '10003', '축구')


In [68]:
cursor.execute("SELECT * FROM money WHERE in_out = 'in';")
in_list = cursor.fetchall()
cursor.execute("SELECT * FROM money WHERE in_out = 'out';")
out_list = cursor.fetchall()

In [69]:
print(in_list)

[('2025-05-12', 'in', '1000'), ('2025-05-12', 'in', '1000'), ('2025-05-12', 'in', '10001')]


In [70]:
from flask import Flask, render_template

app = Flask(__name__)
cursor.execute("SELECT * FROM money WHERE in_out = 'in';")
in_list_en = cursor.fetchall()
cursor.execute("SELECT * FROM money WHERE in_out = 'out';")
out_list_en = cursor.fetchall()
@app.route('/money')
def money_page():
    in_list = in_list_en
    out_list = out_list_en

    return render_template('money_page.html', in_list=in_list, out_list=out_list)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/May/2025 14:15:20] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [12/May/2025 14:15:27] "GET /money HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 14:16:51] "GET /money HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 14:19:17] "GET /money HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 14:19:47] "GET /money HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2025 14:20:08] "GET /money HTTP/1.1" 200 -


In [3]:
from flask import Flask, render_template, request, redirect, url_for
from datetime import date
import sqlite3 

app = Flask(__name__)

DATABASE = "D:/AHN/Codng/python/2025_kimmich.kr_web/coding/DB/money.db"

def get_db_connection():
    """요청마다 새로운 SQLite 연결 생성"""
    connection = sqlite3.connect(DATABASE)
    connection.row_factory = sqlite3.Row  # 행을 딕셔너리처럼 사용 가능
    return connection

@app.route('/money')
def money_page():
    today = date.today()  # 오늘 날짜 가져오기
    connection = get_db_connection()
    cursor = connection.cursor()
    cursor.execute("SELECT rowid, * FROM money WHERE in_out = 'in' AND date = ?", (today,))
    in_list = cursor.fetchall()
    cursor.execute("SELECT rowid, * FROM money WHERE in_out = 'out' AND date = ?", (today,))
    out_list = cursor.fetchall()
    connection.close()

    return render_template('money_page.html', in_list=in_list, out_list=out_list)

@app.route('/add_in', methods=['POST'])
def add_in():
    fname = request.form['fname']
    lname = request.form['lname']
    today = date.today()

    connection = get_db_connection()
    cursor = connection.cursor()
    cursor.execute(f"""
        INSERT INTO money (date, in_out, price, content)
        VALUES ('{today}', 'in', {lname}, '{fname}')
    """)
    connection.commit()
    connection.close()
    return redirect(url_for('money_page'))

@app.route('/add_out', methods=['POST'])
def add_out():
    fname = request.form['fname']
    lname = request.form['lname']
    today = date.today()

    connection = get_db_connection()
    cursor = connection.cursor()
    cursor.execute(f"""
        INSERT INTO money (date, in_out, price, content)
        VALUES ('{today}', 'out', {lname}, '{fname}')
    """)
    connection.commit()
    connection.close()
    return redirect(url_for('money_page'))

@app.route('/delete/<int:rowid>', methods=['POST'])
def delete(rowid):
    connection = get_db_connection()
    cursor = connection.cursor()
    cursor.execute("DELETE FROM money WHERE rowid = ?", (rowid,))
    connection.commit()
    connection.close()
    return redirect(url_for('money_page'))

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/May/2025 11:20:38] "GET /add_in HTTP/1.1" 405 -
